Chris Holden (ceholden@gmail.com) - https://github.com/ceholden

Chapter 4: Importing and using vector data -- the OGR library
==================================================

## 简介

*OGR* 库是 *GDAL* 的伴随库，处理矢量数据功能，包括信息查询、文件转换、多边形要素的栅格化、栅格要素的多边形化等。它支持流行的格式，包括 *ESRI Shapefile*、*Keyhole Markup Language*、*PostGIS* 和 *SpatiaLite*。有关 *OGR* 的详细信息以及它与 *GDAL* 的关系，请参阅这里：[http://trac.osgeo.org/gdal/wiki/FAQGeneral#WhatisthisOGRstuff](http://trac.osgeo.org/gdal/wiki/FAQGeneral#WhatisthisOGRstuff)。

*GDAL*/*OGR* 的作者为 *OGR* 库提供了一个教程，您可以在这里找到：[http://www.gdal.org/ogr_apitut.html](http://www.gdal.org/ogr_apitut.html)。

> 注意：截至 2014 年 8 月 12 日，此教程中使用的 API 似乎超前于当前的 1.11.0 版本。具体来说，他们演示了如何使用 `gdal.OpenEx` 打开矢量文件，这是一项旨在统一库中 *GDAL* 和 *OGR* 部分的更改。
    
> 可以在这里找到 *GDAL 1.9* API 教程的克隆版本：[http://www.compsci.wm.edu/SciClone/documentation/software/geo/gdal-1.9.0/html/ogr/ogr_apitut.html](http://www.compsci.wm.edu/SciClone/documentation/software/geo/gdal-1.9.0/html/ogr/ogr_apitut.html)

在本章中，我们将使用一个包含我在 QGIS 中收集的训练数据的 *ESRI Shapefile*，用于我们一直在处理的示例图像。

## 打开 *ESRI Shapefile*

与 *GDAL* 一样，*OGR* 也抽象了文件格式，以便我们可以对任何格式使用相同的代码。它采用了相同的 *数据集* 对象的概念，我们可以从中获取信息：


(1) Tutorials — GDAL documentation. https://gdal.org/tutorials/index.html.
(2) Mastering GDAL Tools (Full Course Material) - Spatial Thoughts. https://courses.spatialthoughts.com/gdal-tools.html.
(3) Vector API tutorial — GDAL documentation. https://gdal.org/tutorials/vector_api_tut.html.

In [2]:
# Import Python 3 print function
from __future__ import print_function

# Import OGR - 
from osgeo import ogr

# Open the dataset from the file
dataset = ogr.Open('../../example/training_data.shp')
# Make sure the dataset exists -- it would be None if we couldn't open it
if not dataset:
    print('Error: could not open dataset')

读入Shapefile后，我们可以看看它的一些属性:

In [4]:
### Let's get the driver from this file
driver = dataset.GetDriver()
print('Dataset driver is: {n}\n'.format(n=driver.name))

### How many layers are contained in this Shapefile?
layer_count = dataset.GetLayerCount()
print('The shapefile has {n} layer(s)\n'.format(n=layer_count))

### What is the name of the 1 layer?
layer = dataset.GetLayerByIndex(0)
print('The layer is named: {n}\n'.format(n=layer.GetName()))

### What is the layer's geometry? is it a point? a polyline? a polygon?
# First read in the geometry - but this is the enumerated type's value
geometry = layer.GetGeomType()

# So we need to translate it to the name of the enum
geometry_name = ogr.GeometryTypeToName(geometry)
print("The layer's geometry is: {geom}\n".format(geom=geometry_name))

### What is the layer's projection?
# Get the spatial reference
spatial_ref = layer.GetSpatialRef()

# Export this spatial reference to something we can read... like the Proj4
proj4 = spatial_ref.ExportToProj4()
print('Layer projection is: {proj4}\n'.format(proj4=proj4))

### How many features are in the layer?
feature_count = layer.GetFeatureCount()
print('Layer has {n} features\n'.format(n=feature_count))

### How many fields are in the shapefile, and what are their names?
# First we need to capture the layer definition
defn = layer.GetLayerDefn()

# How many fields
field_count = defn.GetFieldCount()
print('Layer has {n} fields'.format(n=field_count))

# What are their names?
print('Their names are: ')
for i in range(field_count):
    field_defn = defn.GetFieldDefn(i)
    print('\t{name} - {datatype}'.format(name=field_defn.GetName(),
                                         datatype=field_defn.GetTypeName()))

Dataset driver is: ESRI Shapefile

The shapefile has 1 layer(s)

The layer is named: training_data

The layer's geometry is: Polygon

Layer projection is: +proj=utm +zone=15 +datum=WGS84 +units=m +no_defs

Layer has 30 features

Layer has 2 fields
Their names are: 
	id - Integer64
	class - String


这个 shapefile 已经使用与我们示例栅格图像相同的投影进行了投影，因此我们不需要重新投影它。不过，您可以使用 [ogr2ogr](http://www.gdal.org/ogr2ogr.html) 命令行应用程序或者通过 [在 Python 中重新投影 shapefile](http://pcjericks.github.io/py-gdalogr-cookbook/projection.html#reproject-a-layer) 来进行投影。

## 与我们的栅格数据集关联

我们刚刚打开的训练数据包含两个字段：
+ ID 字段（整数数据类型）
+ 类别字段（字符串数据类型）

结合 Shapefile 中多边形的固有位置信息，这些字段就足够用于将标签（即整数 ID 和字符串描述）与我们栅格中的信息配对。

然而，为了将我们的矢量数据与栅格像素配对，我们需要一种在空间上对齐数据集的方法。

一种（复杂的）方法是手动遍历矢量图层中的每个多边形，并确定栅格中包含哪些像素。这正是 GIS 软件（例如 ENVI、ArcGIS、QGIS）在进行栅格与矢量配对时所做的工作，比如进行区域统计。

另一种不太复杂的方法是使用感兴趣区域（ROI）图像的概念，其中 ROI 图像中的每个非零像素值对应于来自矢量图层的多边形的栅格表示。在我们的训练数据示例中，大部分值在栅格化表示中都为 0，因为与整个研究区域相比，我们的训练数据样本很小。由于我们为每个多边形分配了整数 ID 字段，我们可以使用这些整数来存储关于哪些多边形属于哪些像素的信息。在这种情况下，我为类别分配了从 1 到 5 的值：

+ 1 - 森林
+ 2 - 水域
+ 3 - 草本植物
+ 4 - 贫瘠地区
+ 5 - 城市区域

为了实现矢量图层的栅格化，我们可以使用 GDAL 命令行实用程序 [gdal_rasterize](http://www.gdal.org/gdal_rasterize.html)，但我们也可以纯粹使用 Python 来使用 GDAL 函数 [gdal.RasterizeLayer](http://gdal.org/gdal__alg_8h.html#adfe5e5d287d6c184aab03acbfa567cb1)。


(1) Python Examples of gdal.RasterizeLayer - ProgramCreek.com. https://www.programcreek.com/python/example/101827/gdal.RasterizeLayer.
(2) Python Examples of osgeo.gdal.RasterizeLayer - ProgramCreek.com. https://www.programcreek.com/python/example/91651/osgeo.gdal.RasterizeLayer.
(3) Use Python to Convert Polygons to Raster with GDAL.RasterizeLayer. https://opensourceoptions.com/use-python-to-convert-polygons-to-raster-with-gdal-rasterizelayer/.
(4) Gdal RasterizeLayer Python - Geographic Information Systems Stack Exchange. https://gis.stackexchange.com/questions/286166/gdal-rasterizelayer-python.

(1) Tutorials — GDAL documentation. https://gdal.org/tutorials/index.html.
(2) Mastering GDAL Tools (Full Course Material) - Spatial Thoughts. https://courses.spatialthoughts.com/gdal-tools.html.
(3) Vector API tutorial — GDAL documentation. https://gdal.org/tutorials/vector_api_tut.html.



####命令行版本——gdal_rasterize

首先，我们需要确定输出栅格的空间范围和像素大小。为此，我将使用 [gdalinfo](http://www.gdal.org/gdalinfo.html)：


(1) gdalinfo — GDAL documentation. https://gdal.org/programs/gdalinfo.html.
(2) python - How to create Raster statistics with GDAL externally .... https://gis.stackexchange.com/questions/208996/how-to-create-raster-statistics-with-gdal-externally.
(3) Gdalinfo Python - February 24, 2024. https://mapscaping.com/gdalinfo-python/.
(4) Python Sample scripts — GDAL documentation. https://gdal.org/api/python_samples.html.

In [7]:
%%bash
# Remember -- "%bash" as seen above just indicates to the IPython notebook that I'm now writing in Bash

# Print out metadata about raster -- we include "-proj4" to print out the Proj4 projection string
gdalinfo -proj4 ../../example/LE70220491999322EDC01_stack.gtif

Couldn't find program: 'bash'


哇，这些信息真是太有用了！

我们将使用关于左上角和右下角坐标的信息：

> 左上角  (  462405.000, 1741815.000) ( 93度21' 3.44"西, 15度45'16.33"北)

> 右下角 (  469905.000, 1734315.000) ( 93度16'51.06"西, 15度41'12.60"北)

这告诉我们我们的左上角 X/Y 和右下角 X/Y 分别为 "462405, 1741815, 469905, 1734315"。我们还可以看到我们的 Landsat 像素为 30x30 米。

投影是 UTM15N，投影字符串为 `'+proj=utm +zone=15 +datum=WGS84 +units=m +no_defs '`。

我们将需要这些信息来使用 `gdal_rasterize`。我们可以打印出 `gdal_rasterize` 的用法，如下所示：

(1) gdal_rasterize — GDAL documentation. https://gdal.org/programs/gdal_rasterize.html.
(2) GDAL Python Tutorial: Reading and Writing Raster Datasets. https://opensourceoptions.com/gdal-python-tutorial-reading-and-writing-raster-datasets/.
(3) Raster API tutorial — GDAL documentation. https://gdal.org/tutorials/raster_api_tut.html.
(4) gdalinfo — GDAL documentation. https://gdal.org/programs/gdalinfo.html.
(5) python - Rasterizing a GDAL layer - Stack Overflow. https://stackoverflow.com/questions/2220749/rasterizing-a-gdal-layer.

In [8]:
%%bash

# Print out the usage
gdal_rasterize --help

Couldn't find program: 'bash'


For better descriptions, see the documentation page [here](http://www.gdal.org/gdal_rasterize.html).

Now let's run the command -- note that we need to rearrange the Upper Left and Lower Right X/Y pairs to be in the "xmin ymin xmax ymax":

In [9]:
%%bash

# Explanation of switches:
# -a ==> write values from the"id" attribute of the shapefile
# -layer ==> the layer name of our shapefile
# -of ==> Output raster file format
# -a_srs ==> output spatial reference system string
# -a_nodata ==> NODATA value for output raster
# -te ==> target extent which matches the raster we want to create the ROI image for
# -tr ==> target resolution, 30 x 30m
# -ot Byte ==> Since we only have values 0 - 5, a Byte datatype is enough

gdal_rasterize -a "id" \
    -l training_data \
    -of "GTiff" \
    -a_srs "+proj=utm +zone=15 +datum=WGS84 +units=m +no_defs" \
    -a_nodata 0 \
    -te 462405 1734315 469905 1741815 \
    -tr 30 30 \
    -ot Byte \
    ../../example/training_data.shp ../../example/training_data.gtif

Couldn't find program: 'bash'


在许多方面，命令行版本比使用Python绑定到GDAL的API进行编程更容易。继续看第二个方法的例子:

#### Pure Python version -- gdal.RasterizeLayer

In [10]:
# Import GDAL
from osgeo import gdal

# First we will open our raster image, to understand how we will want to rasterize our vector
raster_ds = gdal.Open('../../example/LE70220491999322EDC01_stack.gtif', gdal.GA_ReadOnly)

# Fetch number of rows and columns
ncol = raster_ds.RasterXSize
nrow = raster_ds.RasterYSize

# Fetch projection and extent
proj = raster_ds.GetProjectionRef()
ext = raster_ds.GetGeoTransform()

raster_ds = None

# Create the raster dataset
memory_driver = gdal.GetDriverByName('GTiff')
out_raster_ds = memory_driver.Create('../../example/training_data.gtif', ncol, nrow, 1, gdal.GDT_Byte)

# Set the ROI image's projection and extent to our input raster's projection and extent
out_raster_ds.SetProjection(proj)
out_raster_ds.SetGeoTransform(ext)

# Fill our output band with the 0 blank, no class label, value
b = out_raster_ds.GetRasterBand(1)
b.Fill(0)

# Rasterize the shapefile layer to our new dataset
status = gdal.RasterizeLayer(out_raster_ds,  # output to our new dataset
                             [1],  # output to our new dataset's first band
                             layer,  # rasterize this layer
                             None, None,  # don't worry about transformations since we're in same projection
                             [0],  # burn value 0
                             ['ALL_TOUCHED=TRUE',  # rasterize all pixels touched by polygons
                              'ATTRIBUTE=id']  # put raster values according to the 'id' field values
                             )

# Close dataset
out_raster_ds = None

if status != 0:
    print("I don't think it worked...")
else:
    print("Success")

Success


现在我们有了一个工作方法，我们可以检查我们收集了多少像素的训练数据:

## 检查栅格化图层

In [11]:
# Import NumPy for some statistics
import numpy as np

roi_ds = gdal.Open('../../example/training_data.gtif', gdal.GA_ReadOnly)

roi = roi_ds.GetRasterBand(1).ReadAsArray()

# How many pixels are in each class?
classes = np.unique(roi)
# Iterate over all class labels in the ROI image, printing out some information
for c in classes:
    print('Class {c} contains {n} pixels'.format(c=c,
                                                 n=(roi == c).sum()))

Class 0 contains 61400 pixels
Class 1 contains 583 pixels
Class 2 contains 24 pixels
Class 3 contains 223 pixels
Class 4 contains 173 pixels
Class 5 contains 97 pixels


## 总结

现在我们有了感兴趣区域（ROI）图像，我们可以继续使用它将我们标记的多边形与 Landsat 图像中匹配的像素配对，以训练图像分类器。我们将在下一章中继续这一步骤（链接到[网页](chapter_5_classification.html)或[Notebook](chapter_5_classification.ipynb)）。